# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [ ]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [ ]:
from create_tables import create_database
create_database()

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [ ]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [ ]:
song_files = get_files("data/song_data")

In [ ]:
filepath = song_files[0]
filepath

In [ ]:
df = pd.read_json(filepath, lines=True)
df.head()

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
song_data = df[["song_id", "title", "artist_id", "year", "duration"]].values.tolist()[0]
song_data

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
cur.close()
conn.close()

In [ ]:
cur.execute("""
    drop table songs;
""")
conn.commit()


In [ ]:
cur.execute("""
create table if not exists songs(song_id varchar, title varchar not null, artist_id varchar, year int, duration decimal not null, primary key (song_id))
""")
conn.commit()

In [ ]:
cur.execute("""
    insert into songs(song_id, title, artist_id, year, duration) values(%s, %s, %s, %s, %s) on conflict do nothing
""", song_data)
conn.commit()

In [ ]:
cur.execute("""
select * from songs;
""")
conn.commit()
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values.tolist()[0]
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute("""
drop table artists
""")
conn.commit()

In [ ]:
cur.execute("""
create table if not exists artists(artist_id varchar, name varchar not null, location varchar, latitude float, longitude float, primary key (artist_id))
""")
conn.commit()

In [ ]:
cur.execute("""
insert into artists(artist_id, name, location, latitude, longitude) values (%s, %s, %s, %s, %s) on conflict do nothing
""", artist_data)
conn.commit()

In [ ]:
cur.execute("""
select * from artists;
""")
conn.commit()
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files("data/log_data")

In [ ]:
filepath = log_files[0]

In [ ]:
df = pd.read_json(filepath, lines=True)
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = df[df["page"] == "NextSong"]
df.head()

In [ ]:
t = pd.to_datetime(df["ts"], unit="ms")
t.head()

In [ ]:
#  hour, day, week of year, month, year, and weekday
time_data = ([tm.timestamp(), tm.hour, tm.day, tm.week, tm.month, tm.year, tm.weekday()] for tm in t)
column_labels = (["timestamp", "hour", "day", "week_of_year", "month", "year", "weekday"])

In [ ]:
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
cur.execute("""
drop table time
""")
conn.commit()

In [ ]:
cur.execute("""
create table if not exists time(id serial primary key, timestamp varchar, hour int, day int, week_of_year int, month int, year int, weekday int)
""")
conn.commit()

In [ ]:
for i, row in time_df.iterrows():
    cur.execute("""
        insert into time(timestamp, hour, day, week_of_year, month, year, weekday) values(%s,%s,%s,%s,%s,%s,%s)
    """, list(row))
    conn.commit()

In [ ]:
cur.execute("""
select * from time limit 5;
""")
conn.commit()
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df.head(3)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
cur.execute(
"""
    drop table users
""")
conn.commit()

In [ ]:
cur.execute(
"""
    create table if not exists users(user_id int, first_name varchar, last_name varchar, gender varchar, level varchar, primary key(user_id))
""")
conn.commit()

In [ ]:
for i, row in user_df.iterrows():
    cur.execute("""
        insert into users(user_id, first_name, last_name, gender, level) values(%s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET level = EXCLUDED.level
    """, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [ ]:
cur.execute("""
select * from users limit 3;
""")
conn.commit()
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
cur.execute(
"""
    drop table songplays
""")
conn.commit()

In [ ]:
cur.execute(
"""
create table if not exists songplays(start_time varchar not null, user_id int not null, level varchar, song_id varchar, artist_id varchar, session_id int, location varchar, user_agent varchar)
""")
conn.commit()

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute("""
        select s.song_id, a.artist_id
        from songs as s
        join artists as a on a.artist_id = s.artist_id
        where s.title = %s and a.name = %s and s.duration = %s;
    """, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute("""
        insert into songplays(start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) values(%s, %s, %s, %s, %s, %s, %s, %s)
    """, songplay_data)
    conn.commit()


In [ ]:
curs = conn.cursor()
curs.execute("ROLLBACK")
conn.commit()

In [ ]:
cur.execute("""
select * from songplays limit 3;
""")
conn.commit()
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [1]:
%run -i 'etl.py'

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file